Download data

In [ ]:
!git clone https://github.com/yuyongze/movie-sst2.git

In [27]:
%cd movie-sst2

[Errno 2] No such file or directory: 'movie-sst2'
/home/mahendrand/movie-sst2


Load data

In [28]:
import tensorflow as tf
import pandas as pd
from tensorflow.python.lib.io.tf_record import TFRecordWriter

In [29]:
!pwd


/home/mahendrand/movie-sst2


In [30]:

gpu_options = tf.compat.v1.GPUOptions(allow_growth=True) 
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

/home/mahendrand/anaconda3/envs/tf2gpu/lib/python3.8/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [31]:
# fraction of sample pass to the train and test as example
SAMPLE_FRAC = 0.2
# 80% data for training and 20% data for validate
TRAIN_FRAC = 0.8
# load train data from train.csv
train = pd.read_csv('./data/train.csv')
train.reset_index(inplace=True)
# change sentiment label form 'pos' and 'neg' to 1 and 0, which bert model knows
train['sentiment'].replace({'pos':1,'neg':0},inplace=True)


In [32]:
train.head()

,index,text,sentiment
0,0,"Now, I won't deny that when I purchased this o...",0
1,1,"The saddest thing about this ""tribute"" is that...",0
2,2,Last night I decided to watch the prequel or s...,0
3,3,I have to admit that i liked the first half of...,0
4,4,I was not impressed about this film especially...,0


In [33]:
# train set
train_sample = train.sample(frac=SAMPLE_FRAC,random_state=0)
train_select = train_sample.sample(frac= TRAIN_FRAC,random_state=0)
train_csv = train_select.values

# validate set 
validate_select = train_sample.drop(index=train_select.index)
validate_csv = validate_select.values


# load test data , here should be validation set
test = pd.read_csv('./data/test.csv')
test.reset_index(inplace=True)
test['sentiment'].replace({'pos':1,'neg':0},inplace=True)
test_csv = test.sample(frac=SAMPLE_FRAC,random_state=0).values

In [34]:
test.head()

,index,text,sentiment
0,0,"My daughter liked it but I was aghast, that a ...",0
1,1,I... No words. No words can describe this. I w...,0
2,2,this film is basically a poor take on the old ...,0
3,3,"This is a terrible movie, and I'm not even sur...",0
4,4,First of all this movie is a piece of reality ...,1


Build TFRecord

In [35]:
import time
def create_tf_example(features,label):
    """
    Create tf example using features and label

    Args:
        features: list, feature list with format  ['idx','sentence']
        label: string, 

    Return:
        A binary-string of tf example.
        All proto messages can be serialized to a binary-string using the .SerializeToString method.
    """
    tf_example = tf.train.Example(features = tf.train.Features(feature = {
        'idx': tf.train.Feature(int64_list=tf.train.Int64List(value=[features[0]])),
        'sentence': tf.train.Feature(bytes_list=tf.train.BytesList(value=[features[1].encode('utf-8')])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }))
    return tf_example.SerializeToString()

def convert_csv_to_tfrecord(csv, file_name):
    """
    Convert the numpy arryes to tfrecord and write files

    Args:
        csv: numpy arrays, each row feed (features+label)
        file_name: location TFRecord to be saved 
    """
    start_time = time.time()
    writer = TFRecordWriter(file_name)
    for idx,row in enumerate(csv):
        # check the row retionality, raise error when missing value
        try:
            if row is None:
                raise Exception('Row Missing')
            if row[0] is None or row[1] is None or row[2] is None:
                raise Exception('Value Missing')
            if row[1].strip() is '':
                raise Exception('Utterance is empty')
            
            features, label = row[:-1],row[-1]
            example =  create_tf_example(features,label)
            writer.write(example)

        except Exception as inst:
            print(type(inst))
            print(inst.args)
            print(inst)
    writer.close()
    print(f"{file_name}: --- {(time.time() - start_time)} seconds ---")

<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-35-707dd93ba5d7>:38: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if row[1].strip() is '':


In [36]:
train_csv[0,2]

1

In [37]:
convert_csv_to_tfrecord(train_csv, "./data/movie_train.tfrecord")
convert_csv_to_tfrecord(validate_csv, "./data/movie_validate.tfrecord")
convert_csv_to_tfrecord(test_csv, "./data/movie_test.tfrecord")

./data/movie_train.tfrecord: --- 0.2768573760986328 seconds ---
./data/movie_validate.tfrecord: --- 0.07129359245300293 seconds ---
./data/movie_test.tfrecord: --- 0.3689699172973633 seconds ---


In [38]:
import json
# generate exmaple number , save for use in the future 
def generate_json_info(local_file_name,df_train=[],df_val=[],df_test=[]):
    info = {"train_length": len(df_train), "validation_length": len(df_val),
            "test_length": len(df_test)}

    with open(local_file_name, 'w') as outfile:
        json.dump(info, outfile)

generate_json_info('./data/info.json',train_csv,validate_csv,test_csv)

In [39]:
tr_ds = tf.data.TFRecordDataset("data/movie_train.tfrecord")

In [40]:
# Create a description of the features.
feature_spec = {
    'idx': tf.io.FixedLenFeature([], tf.int64),
    'sentence': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}
def parse_example(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, feature_spec)
tr_parse_ds = tr_ds.map(parse_example)
dataset_iterator = iter(tr_parse_ds)

In [41]:
dataset_iterator.get_next()

{'idx': <tf.Tensor: shape=(), dtype=int64, numpy=8383>,
 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 'sentence': <tf.Tensor: shape=(), dtype=string, numpy=b"This movie is difficult to watch in our fast-paced culture of the 21st century, but it is worth it for the messages that it conveys, chiefly the consequences and ramifications of technology upon society, specifically when that technology is used for warfare.<br /><br />This movie presents a full circle cycle of dehumanization and rehumanization as influenced by the advent of technology and the subsequent deconstruction of civilization and therefore serves as a cautionary tale against the misuse of technology, but as the circle completes itself, familiar themes and sentiments pop up again to present self-serving rather than self-destructive ways that humanity may utilize technology.<br /><br />Brilliant for it's time, the picture and sound quality may pose a challenge for some, but as a landmark in the history, developmen

In [42]:
import tensorflow as tf
from transformers import *
from transformers import BertTokenizer, TFBertForSequenceClassification, glue_convert_examples_to_features
from transformers.configuration_bert import BertConfig

In [43]:
tr_ds = tf.data.TFRecordDataset("data/movie_train.tfrecord")
val_ds = tf.data.TFRecordDataset("data/movie_validate.tfrecord")
test_ds = tf.data.TFRecordDataset("data/movie_test.tfrecord")

In [44]:
# Create a description of the features.
feature_spec = {
    'idx': tf.io.FixedLenFeature([], tf.int64),
    'sentence': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}
def parse_example(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    return tf.io.parse_single_example(example_proto, feature_spec)

# convert the encoded string tensor into the separate tensors that will feed into the model
tr_parse_ds = tr_ds.map(parse_example)
val_parse_ds = val_ds.map(parse_example)
test_parse_ds =  test_ds.map(parse_example)

In [45]:
def clean_string(features):
    revised_sentence = tf.strings.regex_replace(features['sentence'], "\.\.\.", "", replace_global=True)
    revised_sentence = tf.strings.regex_replace(revised_sentence, "\\'", "'", replace_global=True)
    revised_sentence = tf.strings.regex_replace(revised_sentence, "\\n", "", replace_global=True)
    features['sentence'] = revised_sentence
    return features

In [46]:
tr_clean_ds = tr_parse_ds.map(lambda features: clean_string(features))
val_clean_ds = val_parse_ds.map(lambda features: clean_string(features))
test_clean_ds =  test_parse_ds.map(lambda features: clean_string(features))

In [47]:
BATCH_SIZE = 8

EVAL_BATCH_SIZE = BATCH_SIZE * 2

# XLA is the optimizing compiler for machine learning
# It can potentially increase speed by 15% with no source code changes
USE_XLA = False

# mixed precision results on https://github.com/huggingface/transformers/tree/master/examples
# Mixed precision can help to speed up training time
USE_AMP = False

In [48]:
tf.config.optimizer.set_jit(USE_XLA)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": USE_AMP})

In [49]:
# Steps is determined by the number of examples
import json
with open('data/info.json') as json_file:
    data_info = json.load(json_file)
    
train_examples = data_info['train_length']
valid_examples = data_info['validation_length']
test_examples = data_info['test_length']

train_examples, valid_examples, test_examples

(4000, 1000, 5000)

In [50]:
from ipywidgets import IntProgress

In [51]:
# Load tokenizer and model from pretrained model/vocabulary. Specify the number of labels to classify (2+: classification, 1: regression)
num_labels = 2 
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dropout_37', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
for example in tr_clean_ds:
    print(example)

{'idx': <tf.Tensor: shape=(), dtype=int64, numpy=8383>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'sentence': <tf.Tensor: shape=(), dtype=string, numpy=b"This movie is difficult to watch in our fast-paced culture of the 21st century, but it is worth it for the messages that it conveys, chiefly the consequences and ramifications of technology upon society, specifically when that technology is used for warfare.<br /><br />This movie presents a full circle cycle of dehumanization and rehumanization as influenced by the advent of technology and the subsequent deconstruction of civilization and therefore serves as a cautionary tale against the misuse of technology, but as the circle completes itself, familiar themes and sentiments pop up again to present self-serving rather than self-destructive ways that humanity may utilize technology.<br /><br />Brilliant for it's time, the picture and sound quality may pose a challenge for some, but as a landmark in the history, development,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
import time
start_time = time.time()
train_dataset = glue_convert_examples_to_features(examples=tr_clean_ds, tokenizer=tokenizer
                                                  , max_length=128, task='sst-2',
                                                  label_list=['0','1']
                                                  )
print(f"---{time.time()-start_time} seconds---")

---21.262656927108765 seconds---


In [32]:
import time
start_time = time.time()
valid_dataset = glue_convert_examples_to_features(examples=val_clean_ds, tokenizer=tokenizer
                                                  , max_length=128, task='sst-2'
                                                  , label_list =['0', '1'])
print(f"---{time.time()-start_time} seconds---")

---5.172550916671753 seconds---


In [33]:
train_dataset = train_dataset.shuffle(train_examples).batch(BATCH_SIZE).repeat(-1)

valid_dataset = valid_dataset.batch(EVAL_BATCH_SIZE)

In [34]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

if USE_AMP:
    # loss scaling is currently required when using mixed precision
    opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, 'dynamic')

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [35]:
train_steps = train_examples//BATCH_SIZE
valid_steps = valid_examples//EVAL_BATCH_SIZE

In [36]:
# GPU USAGE
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

Num GPUs Available:  4


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:2', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:3', device_type='XLA_GPU')]

In [37]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [39]:
history = model.fit(train_dataset, epochs=3, steps_per_epoch=train_steps,
                    validation_data=valid_dataset, validation_steps=valid_steps)

Epoch 1/3
500/500 [==============================] - 143s 287ms/step - loss: 0.4243 - accuracy: 0.8062 - val_loss: 0.3529 - val_accuracy: 0.8619
Epoch 2/3
500/500 [==============================] - 141s 282ms/step - loss: 0.2239 - accuracy: 0.9190 - val_loss: 0.3980 - val_accuracy: 0.8498
Epoch 3/3
500/500 [==============================] - 141s 283ms/step - loss: 0.1299 - accuracy: 0.9563 - val_loss: 0.4492 - val_accuracy: 0.8458


In [41]:
import time
start_time = time.time()
test_dataset = glue_convert_examples_to_features(examples=test_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2'
                                                  , label_list =['0', '1'])
print(f"---{time.time()-start_time} seconds---")

---25.01020097732544 seconds---


In [42]:
test_dataset = test_dataset.batch(EVAL_BATCH_SIZE)

In [43]:
model.evaluate(test_dataset)

313/313 [==============================] - 226s 721ms/step - loss: 0.3220 - accuracy: 0.8926


[0.3220312297344208, 0.8925999999046326]